# 08.01 - Introduccion a GeoPandas

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 30 min

---

## Que aprenderemos?

- Que es GeoPandas y GeoDataFrame
- Tipos de geometria (Point, LineString, Polygon)
- Crear y leer datos espaciales
- Sistemas de coordenadas (CRS)
- Visualizacion basica de mapas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Verificar instalacion
try:
    import geopandas as gpd
    from shapely.geometry import Point, LineString, Polygon
    print(f"geopandas version: {gpd.__version__}")
    GEOPANDAS_AVAILABLE = True
except ImportError:
    print("geopandas no instalado.")
    print("Instalar con: pip install geopandas")
    GEOPANDAS_AVAILABLE = False

geopandas no instalado.
Instalar con: pip install geopandas


---

## 1. Que es GeoPandas?

GeoPandas extiende pandas para trabajar con datos geograficos:

- **GeoDataFrame** = DataFrame + columna de geometria
- Operaciones espaciales (interseccion, buffer, etc.)
- Lectura/escritura de formatos GIS (Shapefile, GeoJSON, etc.)
- Visualizacion de mapas integrada

---

## 2. Tipos de geometria

In [2]:
if GEOPANDAS_AVAILABLE:
    # Point: ubicacion (estacion de bici)
    point = Point(-3.7038, 40.4168)  # Madrid (lon, lat)
    print(f"Point: {point}")
    print(f"Coordenadas: x={point.x}, y={point.y}")

In [3]:
if GEOPANDAS_AVAILABLE:
    # LineString: ruta, calle
    line = LineString([(-3.71, 40.42), (-3.70, 40.41), (-3.69, 40.415)])
    print(f"LineString: {line}")
    print(f"Longitud: {line.length:.4f} grados")

In [4]:
if GEOPANDAS_AVAILABLE:
    # Polygon: area (distrito, parque)
    polygon = Polygon([(-3.71, 40.41), (-3.69, 40.41), 
                       (-3.69, 40.43), (-3.71, 40.43)])
    print(f"Polygon: {polygon}")
    print(f"Area: {polygon.area:.6f} grados^2")

In [5]:
if GEOPANDAS_AVAILABLE:
    # Visualizar geometrias
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Point
    gpd.GeoSeries([point]).plot(ax=axes[0], color='red', markersize=100)
    axes[0].set_title('Point')
    
    # LineString
    gpd.GeoSeries([line]).plot(ax=axes[1], color='blue', linewidth=2)
    axes[1].set_title('LineString')
    
    # Polygon
    gpd.GeoSeries([polygon]).plot(ax=axes[2], color='green', alpha=0.5, edgecolor='black')
    axes[2].set_title('Polygon')
    
    plt.tight_layout()
    plt.show()

---

## 3. Crear un GeoDataFrame

In [6]:
if GEOPANDAS_AVAILABLE:
    # Datos de estaciones de bici en Madrid
    stations_data = {
        'name': ['Sol', 'Atocha', 'Cibeles', 'Retiro', 'Gran Via'],
        'bikes': [15, 22, 8, 12, 18],
        'docks': [30, 35, 25, 28, 32],
        'lat': [40.4168, 40.4065, 40.4197, 40.4153, 40.4203],
        'lon': [-3.7038, -3.6893, -3.6921, -3.6844, -3.7065]
    }
    
    df = pd.DataFrame(stations_data)
    df

In [7]:
if GEOPANDAS_AVAILABLE:
    # Convertir a GeoDataFrame
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    
    gdf = gpd.GeoDataFrame(
        df, 
        geometry=geometry,
        crs="EPSG:4326"  # WGS84 (lat/lon)
    )
    
    gdf

In [8]:
if GEOPANDAS_AVAILABLE:
    # Informacion del GeoDataFrame
    print(f"Tipo: {type(gdf)}")
    print(f"CRS: {gdf.crs}")
    print(f"Columna geometria: {gdf.geometry.name}")
    print(f"Bounds: {gdf.total_bounds}")  # [minx, miny, maxx, maxy]

### Metodo alternativo: gpd.points_from_xy

In [9]:
if GEOPANDAS_AVAILABLE:
    # Forma mas directa
    gdf2 = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['lon'], df['lat']),
        crs="EPSG:4326"
    )
    gdf2.head()

---

## 4. Visualizar en mapa

In [10]:
if GEOPANDAS_AVAILABLE:
    # Mapa simple
    fig, ax = plt.subplots(figsize=(10, 8))
    
    gdf.plot(
        ax=ax,
        column='bikes',      # Color por variable
        cmap='RdYlGn',       # Colormap
        markersize=100,
        legend=True,
        legend_kwds={'label': 'Bicis disponibles'}
    )
    
    # Etiquetas
    for idx, row in gdf.iterrows():
        ax.annotate(
            row['name'],
            xy=(row.geometry.x, row.geometry.y),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=9
        )
    
    ax.set_title('Estaciones de BiciMAD', fontsize=14)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')
    
    plt.show()

---

## 5. Sistemas de coordenadas (CRS)

In [11]:
if GEOPANDAS_AVAILABLE:
    print(f"CRS actual: {gdf.crs}")
    print(f"Es geografico: {gdf.crs.is_geographic}")

In [12]:
if GEOPANDAS_AVAILABLE:
    # Reproyectar a metros (UTM zona 30N para Madrid)
    gdf_utm = gdf.to_crs("EPSG:25830")
    
    print(f"CRS nuevo: {gdf_utm.crs}")
    print(f"Es proyectado: {gdf_utm.crs.is_projected}")

In [13]:
if GEOPANDAS_AVAILABLE:
    # Comparar coordenadas
    print("Coordenadas WGS84 (grados):")
    print(gdf[['name', 'geometry']].head(2))
    
    print("\nCoordenadas UTM (metros):")
    print(gdf_utm[['name', 'geometry']].head(2))

---

## 6. Leer datos espaciales

In [14]:
if GEOPANDAS_AVAILABLE:
    # Dataset de ejemplo incluido en geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    world.head()

In [15]:
if GEOPANDAS_AVAILABLE:
    # Mapa del mundo
    fig, ax = plt.subplots(figsize=(15, 8))
    
    world.plot(
        ax=ax,
        column='pop_est',
        cmap='YlOrRd',
        legend=True,
        legend_kwds={'label': 'Poblacion'}
    )
    
    ax.set_title('Poblacion Mundial', fontsize=14)
    ax.axis('off')
    
    plt.show()

In [16]:
if GEOPANDAS_AVAILABLE:
    # Filtrar Europa
    europe = world[world['continent'] == 'Europe']
    
    fig, ax = plt.subplots(figsize=(12, 10))
    europe.plot(ax=ax, column='pop_est', cmap='Blues', 
                legend=True, edgecolor='white', linewidth=0.5)
    ax.set_title('Europa - Poblacion', fontsize=14)
    ax.set_xlim(-25, 45)
    ax.set_ylim(35, 72)
    ax.axis('off')
    plt.show()

---

## 7. Guardar datos espaciales

In [17]:
if GEOPANDAS_AVAILABLE:
    # Guardar como GeoJSON
    # gdf.to_file('stations.geojson', driver='GeoJSON')
    
    # Guardar como Shapefile
    # gdf.to_file('stations.shp')
    
    # Guardar como GeoPackage (recomendado)
    # gdf.to_file('stations.gpkg', driver='GPKG')
    
    print("Formatos de guardado:")
    print("  GeoJSON: gdf.to_file('file.geojson', driver='GeoJSON')")
    print("  Shapefile: gdf.to_file('file.shp')")
    print("  GeoPackage: gdf.to_file('file.gpkg', driver='GPKG')")

---

## Resumen

| Concepto | Descripcion |
|----------|-------------|
| GeoDataFrame | DataFrame con columna geometry |
| Point | Ubicacion (estacion, POI) |
| LineString | Ruta, calle, recorrido |
| Polygon | Area (distrito, parque) |
| CRS | Sistema de coordenadas |

**CRS comunes:**
- EPSG:4326 - WGS84 (lat/lon, GPS)
- EPSG:25830 - UTM 30N (metros, Espana)
- EPSG:3857 - Web Mercator (Google Maps)

---

**Anterior:** [07.04 - Dashboard Matplotlib](../07_visualization/07_04_matplotlib_dashboard.ipynb)  
**Siguiente:** [08.02 - Operaciones Espaciales](08_02_spatial_operations.ipynb)